In [ ]:
import pandas as pd
import numpy as np
import shutil
import os
import random

In [ ]:
df = pd.read_json("data/annotations.json")

In [ ]:
df.head()

In [ ]:
df["general_figure_info"][10052]["x_axis"]["major_labels"]["values"]

In [ ]:
def is_number(s):

    try:
        float(s)
        return True
    except ValueError:
        return False
    
def variable_type(str):

    if is_number(str): return "quantitative"
    return "nominal"
    
def x_variable_type(dict):
    return variable_type(dict["x_axis"]["major_labels"]["values"][0])

def y_variable_type(dict):
    return variable_type(dict["y_axis"]["major_labels"]["values"][0])

def get_xml(dataframe, index):

    mark = dataframe["mark"][index]

    x_type = dataframe["x_type"][index]
    y_type = dataframe["y_type"] [index]

    x_field = "<field>" + dataframe["x_field"][index] + "</field>"
    y_field = "<field>" + dataframe["y_field"][index] + "</field>"

    x_field = x_field.replace(",","")
    y_field = y_field.replace(",","")

    return "<mark>" + mark + "</mark><x><type>" + x_type + "</type>" + x_field + "</x><y><type>" + y_type + "</type>" + y_field + "</y>"

def erase_files(path):

    for filename in os.listdir(path):

        file_path = os.path.join(path, filename)

        try:
            if os.path.isfile(file_path): os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

In [ ]:
mark_types = df["type"]

x_types = df["general_figure_info"].apply(x_variable_type)
y_types = df["general_figure_info"].apply(y_variable_type)

In [ ]:
x_fields = [df["general_figure_info"][i]["x_axis"]["label"]["text"] for i in range(len(df))]
y_fields = [df["general_figure_info"][i]["y_axis"]["label"]["text"] for i in range(len(df))]

In [ ]:
df2 = pd.DataFrame()

df2["mark"] = mark_types
df2.replace({"hbar_categorical": "bar2", "vbar_categorical": "bar", "dot_line": "point"}, inplace = True)

df2["x_type"] = x_types
df2["y_type"] = y_types

df2["x_field"] = x_fields
df2["y_field"] = y_fields

In [ ]:
df2.head()

In [ ]:
def erase_files(path):

    for filename in os.listdir(path):

        file_path = os.path.join(path, filename)

        try:
            if os.path.isfile(file_path): os.remove(file_path)
        except Exception as e:
            print(f"Error deleting {file_path}: {e}")

In [ ]:
def generate_dataset(n, path, start):

    np.random.seed(42)
    random.seed(42)

    erase_files(path)

    indexes = list(df2.sample(n).index)
    metadata = [['file_name','text']]
    file_names = []

    for i, index in enumerate(indexes):
        metadata.append([str(i + start) + ".png", get_xml(df2,index)])
        file_names.append(str(index) + ".png")

    np.savetxt(path + "/metadata.csv", metadata, delimiter = ',', fmt = '% s')

    # --------------------------------------- move files ----------------------------------------------

    source_directory = "plotqa/png"
    destination_directory = path

    for i, file_name in enumerate(file_names):
        
        source_path = os.path.join(source_directory, file_name)
        destination_path = os.path.join(destination_directory, str(i + start) + ".png")

        if os.path.exists(source_path):
            try:
                shutil.copy(source_path, destination_path)
            except IOError as e:
                print(f"Failed to copy {file_name}: {e}")
        else:
            print(f"File {file_name} does not exist in the source directory.")


In [ ]:
generate_dataset(n = 100, path = "dataset_plotqa/test", start = 0)